In [20]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

In [32]:
frac = pd.read_csv("/home/hocine/myProjs/students/Taye/geo/2_purity/output/epic/epic_c1.txt", index_col=0, sep='\t')
frac.index = frac.index.str.replace(".", "-", regex=False)
frac.head()

,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NKcells,otherCells,cluster
sampleID,,,,,,,,,
GSM437231,0.0001,0.0374,0.0000,0.0000,0.3719,0.0000,0.0000,0.5906,1
GSM437232,0.0000,0.0009,0.0329,0.0000,0.1896,0.0000,0.0000,0.7766,1
GSM437233,0.0000,0.0547,0.0000,0.0000,0.1984,0.0174,0.0000,0.7296,1
GSM437234,0.0000,0.0627,0.0004,0.0000,0.4163,0.0613,0.0025,0.4567,1
GSM437235,0.0000,0.0290,0.1412,0.0896,0.1802,0.0114,0.0010,0.5475,1


In [33]:
cd8 = frac["CD8_Tcells"]
cd4 = frac["CD4_Tcells"]
purity = frac["otherCells"]
bcells = frac["Bcells"]
cafs = frac["CAFs"]

In [34]:
expr = pd.read_csv("/home/hocine/myProjs/students/Taye/geo/output/GSE17538/L_2.csv", index_col=0)
expr.head()

,GSM437231,GSM437232,GSM437233,GSM437234,GSM437235,GSM437236,GSM437237,GSM437238,GSM437240,GSM437242,...,GSM437301,GSM437302,GSM437304,GSM437305,GSM437316,GSM437318,GSM437319,GSM437320,GSM437321,GSM437322
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000137332,0.043187,0.016544,0.025969,-0.000306,0.058144,-0.028637,0.019108,0.014277,-0.037956,0.014625,...,-0.057895,-0.013147,-0.021376,-0.014492,-0.108447,-0.006067,-0.012753,-0.015205,-0.019435,-0.001465
ENSG00000049541,-0.004307,-0.049242,-0.083049,0.018892,0.032207,0.049654,0.096128,0.008858,0.027722,0.008471,...,0.021545,0.057575,-0.146692,-0.030996,-0.186230,0.059584,-0.022623,-0.072351,-0.005538,-0.025853
ENSG00000125618,-0.002297,0.017080,0.037287,-0.012410,0.029329,0.005290,0.000698,0.016571,0.001260,-0.008227,...,0.002272,-0.020283,-0.020861,-0.018644,-0.097792,-0.023813,-0.036731,-0.004504,-0.033760,0.019904
ENSG00000290147,-0.003341,0.028852,0.033260,0.026508,0.080552,0.040571,-0.002301,0.021548,0.003842,0.025699,...,-0.094929,-0.117526,-0.112409,-0.127678,-0.072804,-0.077475,-0.004015,-0.138417,-0.113561,-0.130479
ENSG00000182179,-0.053050,0.077173,0.030088,-0.011788,0.034329,-0.042381,-0.014391,0.033908,-0.063383,0.033354,...,-0.033495,-0.024261,0.014134,-0.051937,-0.121803,-0.039302,0.054079,0.003244,-0.011004,0.008826


In [35]:
if set(frac.index).issubset(set(expr.columns)):
    expr_sxg = expr.T
else:
    expr_sxg = expr  # already samples x genes

In [36]:
common = expr_sxg.index.intersection(frac.index)
expr_sxg = expr_sxg.loc[common]
cd8 = cd8.loc[common]
cd4 = cd4.loc[common]

In [37]:
onco_tsg = pd.read_csv("../../../1_deg/output/degs_tsg_onco.csv", index_col=0)
oncogenes = onco_tsg[onco_tsg["is_onco"]==True].index.to_list()
tsgs = onco_tsg[onco_tsg["is_tsg"]==True].index.to_list()

onco_tsg.head()

,logFC,adj.P.Val,hgnc_symbol,is_tsg,is_onco
Ensembl_ID,,,,,
ENSG00000163629,2.014847,2.085728e-06,PTPN13,True,False
ENSG00000023445,1.957911,2.964434e-07,BIRC3,True,True
ENSG00000134982,1.934692,1.799402e-10,APC,True,False
ENSG00000126778,1.887040,2.768037e-04,SIX1,False,True
ENSG00000119508,1.853774,2.032135e-05,NR4A3,False,True


In [38]:
genes = [g for g in (oncogenes + tsgs) if g in expr_sxg.columns]
print("Genes found:", len(genes), "Missing:", len(set(oncogenes+tsgs) - set(genes)))

def corr_table(cell_series, cell_name):
    rows = []
    for g in genes:
        x = cell_series.values
        y = expr_sxg[g].values
        # drop NaNs pairwise
        mask = np.isfinite(x) & np.isfinite(y)
        rho, p = spearmanr(x[mask], y[mask])
        onco = bool(onco_tsg.loc[g, "is_onco"])
        tsg  = bool(onco_tsg.loc[g, "is_tsg"])
        gene_type = "Oncogene/TSG" if (onco and tsg) else ("Oncogene" if onco else ("TSG" if tsg else "Other"))

        rows.append((g, gene_type, cell_name, rho, p))
    out = pd.DataFrame(rows, columns=["gene", "type", "cell", "spearman_rho", "pval"])
    out["qval_fdr"] = multipletests(out["pval"], method="fdr_bh")[1]
    return out.sort_values(["qval_fdr", "pval"])

res_cd8 = corr_table(cd8, "CD8")
res_cd4 = corr_table(cd4, "CD4")

res = pd.concat([res_cd8, res_cd4], ignore_index=True)
res = res.drop_duplicates()
res.to_csv("CD4_CD8_gene_correlations.csv", index=False)
res.head(91)


Genes found: 91 Missing: 0


,gene,type,cell,spearman_rho,pval,qval_fdr
0,ENSG00000115760,Oncogene,CD8,-0.648277,0.000001,0.000118
1,ENSG00000164985,Oncogene,CD8,-0.636642,0.000003,0.000118
2,ENSG00000178691,Oncogene/TSG,CD8,-0.617998,0.000006,0.000139
4,ENSG00000181163,Oncogene,CD8,-0.593956,0.000017,0.000309
5,ENSG00000157554,Oncogene,CD8,-0.579587,0.000030,0.000412
...,...,...,...,...,...,...
96,ENSG00000124795,Oncogene,CD4,-0.574282,0.000037,0.000553
97,ENSG00000134899,TSG,CD4,-0.570702,0.000043,0.000553
98,ENSG00000164985,Oncogene,CD4,-0.563138,0.000056,0.000605
99,ENSG00000143970,TSG,CD4,-0.561546,0.000060,0.000605


In [39]:
from scipy.stats import zscore

def gene_set_score(df_sxg, gene_list):
    genes_present = [g for g in gene_list if g in df_sxg.columns]
    Z = df_sxg[genes_present].apply(zscore, axis=0, nan_policy="omit")
    return Z.mean(axis=1)

onc_score = gene_set_score(expr_sxg, oncogenes)
tsg_score = gene_set_score(expr_sxg, tsgs)

rho_cd8_onc, p_cd8_onc = spearmanr(cd8, onc_score, nan_policy="omit")
rho_cd8_tsg, p_cd8_tsg = spearmanr(cd8, tsg_score, nan_policy="omit")
rho_cd4_onc, p_cd4_onc = spearmanr(cd4, onc_score, nan_policy="omit")
rho_cd4_tsg, p_cd4_tsg = spearmanr(cd4, tsg_score, nan_policy="omit")

print("CD8 vs oncogene score:", rho_cd8_onc, p_cd8_onc)
print("CD8 vs TSG score:",     rho_cd8_tsg, p_cd8_tsg)
print("CD4 vs oncogene score:", rho_cd4_onc, p_cd4_onc)
print("CD4 vs TSG score:",     rho_cd4_tsg, p_cd4_tsg)


CD8 vs oncogene score: -0.45447387682048374 0.0017115498405668472
CD8 vs TSG score: -0.58365267925419 2.5730458658139242e-05
CD4 vs oncogene score: -0.47861685010242416 0.000881859030086356
CD4 vs TSG score: -0.549967955968918 9.098870523944982e-05
